## Setup libraries

In [1]:
from fmi.fmi.explore import *
from fmi.fmi.preprocessing import *
from fmi.fmi.pipeline import *
from fmi.fmi.retinanet import *

/home/karrrrrrrr/miniconda3/envs/fastmed/lib/python3.9/site-packages/kornia/augmentation/augmentation.py:1872: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  warnings.warn(


In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

/home/karrrrrrrr/miniconda3/envs/fastmed/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [1]:
from fastai.callback.all import *
from fastai.vision.all import *
from fastai.medical.imaging import *

/home/karrrrrrrr/miniconda3/envs/fastmed/lib/python3.9/site-packages/kornia/augmentation/augmentation.py:1872: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  warnings.warn(


## Get Data

In [2]:
path = Path('/media/veracrypt1/Data Science/SIIM-Imgs/png')
stage = 'test'

df = pd.read_pickle('src/test_ready.pkl')
sub_df = pd.read_csv('src/sample_submission.csv')

/home/karrrrrrrr/miniconda3/envs/fastmed/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Remove the image file extension

In [6]:
df['id'] = df['id'].apply(lambda x: x[:-4])

Initialize BBoxes & label_lists:

In [7]:
df['resized_boxes'] = [[[0., 0., 1., 1.]] for _ in range(df.shape[0])]
df['labels_list'] = [['Negative'] for _ in range(df.shape[0])]

In [8]:
df.head()

,id,scalar,resized_boxes,labels_list
0,fdbaf93d6c5b,0.415584,"[[0.0, 0.0, 1.0, 1.0]]",[Negative]
1,c6fa6b9b8093,0.390542,"[[0.0, 0.0, 1.0, 1.0]]",[Negative]
2,e5e44940be7a,0.390542,"[[0.0, 0.0, 1.0, 1.0]]",[Negative]
3,a0f473f71878,0.361582,"[[0.0, 0.0, 1.0, 1.0]]",[Negative]
4,3d56a237b6be,0.300469,"[[0.0, 0.0, 1.0, 1.0]]",[Negative]


Get the paths of the image files

In [10]:
items = get_image_files(path/stage)

Get functions for the dataloaders

In [9]:
def get_bbox(fn):
    boxes = df.loc[df['id'] == fn.stem, 'resized_boxes'].values[0]
    return boxes
    
def get_bbox_label(fn): ## for bbox regressor
    labels = df.loc[df['id'] == fn.stem, 'labels_list'].values[0]
    return labels

def get_label(fn): ## for classifier
    return df.loc[df['id'] == fn.stem, 'label'].values[0]

def get_items(noop):
    return items

def get_study(fn):
    return fn.parent.parent.name + '_study'

In [14]:
sub_df.head()

,id,PredictionString
0,00188a671292_study,negative 1 0 0 1 1
1,004bd59708be_study,negative 1 0 0 1 1
2,00508faccd39_study,negative 1 0 0 1 1
3,006486aa80b2_study,negative 1 0 0 1 1
4,00655178fdfc_study,negative 1 0 0 1 1


## Get the sizes, the images got trained on

Test for single item

In [16]:
item = items[0]
img = Image.open(item)
df.loc[df.id == item.stem, 'training_width'] = img.size[0]
df.loc[df.id == item.stem, 'training_height'] = img.size[1]

Add image sizes to dataframe for all images

In [17]:
for item in items:
    img = Image.open(item)
    df.loc[df.id == item.stem, 'training_width'] = img.size[0]
    df.loc[df.id == item.stem, 'training_height'] = img.size[1]  
    
df[['training_width','training_height']] = df[['training_width','training_height']].astype(int)

In [19]:
df.head()

,id,scalar,resized_boxes,labels_list,training_width,training_height
0,fdbaf93d6c5b,0.415584,"[[0.0, 0.0, 1.0, 1.0]]",[Negative],1024,934
1,c6fa6b9b8093,0.390542,"[[0.0, 0.0, 1.0, 1.0]]",[Negative],1024,1068
2,e5e44940be7a,0.390542,"[[0.0, 0.0, 1.0, 1.0]]",[Negative],1024,1068
3,a0f473f71878,0.361582,"[[0.0, 0.0, 1.0, 1.0]]",[Negative],1024,838
4,3d56a237b6be,0.300469,"[[0.0, 0.0, 1.0, 1.0]]",[Negative],1024,841


## Restore Classifier

In [21]:
learn_C = load_learner('classifier_xres50.pkl', cpu = False)

Predict single image

In [23]:
pred = learn_C.predict(items[0])
pred

('Typical',
 tensor(3),
 tensor([5.9027e-04, 2.7610e-02, 5.1239e-04, 9.7129e-01]))

Get the confidence the learner predicted the label with

In [24]:
def get_conf(pred):
    return round(F.softmax(pred[2], dim = -1).max().item(), 5 )

In [25]:
get_conf(pred)

0.46581

Initialize the column for predictions

In [26]:
sub_df['PredictionString'] = ''

In [27]:
sub_df.head()

,id,PredictionString
0,00188a671292_study,
1,004bd59708be_study,
2,00508faccd39_study,
3,006486aa80b2_study,
4,00655178fdfc_study,
...,...,...
2472,46719b856de1_image,
2473,31c07523a69a_image,
2474,f77d7d1aebab_image,
2475,ccc5b63ca96d_image,


## Predict on study level

Test for single image

In [31]:
fn = items[0]

pred = learn_C.predict(fn)
label_pred = pred[0]
conf_pred = get_conf(pred)

' '.join([label_pred, str(conf_pred), '0 0 1 1'])

'Typical 0.46581 0 0 1 1'

In [32]:
label_pred.lower(), conf_pred

('typical', 0.46581)

## Predict all test studies and add result to submission dataframe

In [33]:
for fn in items:
    pred = learn_C.predict(fn)
    label_pred = pred[0]
    conf_pred = get_conf(pred)
    
    if sub_df.loc[sub_df.id == get_study(fn), 'PredictionString'].item() == '':
        stump = ''
    else:
        stump = ' '
    sub_df.loc[sub_df.id == get_study(fn), 'PredictionString'] += ' '.join([stump, label_pred.lower(), str(conf_pred), '0 0 1 1'])

Check result

In [34]:
sub_df.head()

,id,PredictionString
0,00188a671292_study,negative 0.28695 0 0 1 1
1,004bd59708be_study,typical 0.47415 0 0 1 1
2,00508faccd39_study,atypical 0.27307 0 0 1 1
3,006486aa80b2_study,typical 0.38724 0 0 1 1
4,00655178fdfc_study,negative 0.44799 0 0 1 1


## Restore RetinaLearner

In [35]:
def _retinanet_split(m): 
    return L(m.encoder,nn.Sequential(m.c5top6, m.p6top7, m.merges, m.smoothers, m.classifier, m.box_regressor)).map(params)

In [36]:
learn = load_learner('bbox_xres50.pkl', cpu = False)
learn.to('cuda');

In [37]:
scales = [1,2**(-1/3), 2**(-2/3)]
ratios = [1/2,1,2]

## Create (from the learner) expected transformations

In [38]:
size = 256
sizes = (size,int(size * 1.16))

type_tfms = [PILImage.create]
item_tfms = [Resize(sizes, method = ResizeMethod.Squish), ToTensor()]

type_pipe = Pipeline(type_tfms)
item_pipe = Pipeline(item_tfms)
norm = Normalize.from_stats(*imagenet_stats)
i2f = IntToFloatTensor()

## Batchify images and move to GPU

In [39]:
batches = []
batch = []
outs = []
#inps = []
k = 0
for im in items:
    batch.append(item_pipe(type_pipe(im)))
    k += 1
    if len(batch) == 32:
        batches.append(torch.cat([norm(i2f(b.cuda())) for b in batch]))
        batch = []
        k = 0
## append the last batch if the length of the item list is not dividable by the batch size
if len(batch) != 32:
    batches.append(torch.cat([norm(i2f(b.cuda())) for b in batch]))

## Predict on image level

In [40]:
with torch.no_grad():
    for i,b in enumerate(batches):
        outs.append(learn.model.eval()(b))
        #inps.append(b)

## Structure predictions

In [41]:
class_preds = []
bbox_preds = []
sizes = []
for batch in outs:
    class_preds += batch[0]
    bbox_preds += batch[1]
    sizes += [batch[2]] * len(batch[0])
    
raw_preds = list(zip(items, class_preds, bbox_preds, sizes))

In [42]:
raw_preds[0]

(Path('/media/veracrypt1/Data Science/SIIM-Imgs/png/test/7518068f3699/0cece1a81423/c0a746ef9f9f.png'),
 tensor([[-4.4704, -4.1201, -6.2670, -4.0257, -6.2670],
         [-4.5654, -4.4382, -6.2670, -4.1564, -6.2670],
         [-4.9393, -4.5925, -6.2670, -4.1048, -6.2670],
         ...,
         [-4.1400, -4.1813, -4.5298, -4.1231, -4.5298],
         [-4.1698, -4.2884, -4.5298, -4.1468, -4.5298],
         [-4.2640, -4.3619, -4.5298, -4.2831, -4.5298]], device='cuda:0'),
 tensor([[-0.0109, -0.6035,  0.2042,  0.2886],
         [ 0.5032,  0.3609, -0.4604, -0.3679],
         [ 0.6059,  0.5376,  0.0813,  0.0566],
         ...,
         [ 0.0545,  0.0374, -0.0985,  0.0681],
         [ 0.0450,  0.0485, -0.0371,  0.1174],
         [ 0.0522, -0.0492, -0.0035,  0.1594]], device='cuda:0'),
 [[32, 37], [64, 74], [8, 10], [4, 5], [2, 3]])

## Non-max-supression

In [43]:
def nms(boxes, scores, thresh=0.3):
    idx_sort = scores.argsort(descending=True)  ## sort the indices of scores such that the index of the element with the highest value is in front
    boxes, scores = boxes[idx_sort], scores[idx_sort]  ## sort the boxes and scores in above fashion
    to_keep, indexes = [], torch.LongTensor(range_of(scores))
    while len(scores) > 0:
        to_keep.append(idx_sort[indexes[0]])
        iou_vals = IoU_values(boxes, boxes[:1]).squeeze()
        mask_keep = iou_vals < thresh
        if len(mask_keep.nonzero()) == 0: break
        boxes, scores, indexes = boxes[mask_keep], scores[mask_keep], indexes[mask_keep]
    return LongTensor(to_keep)

## Transform predicted bounding box to target format
The target bounding boxes were scaled to a height/width of 2 between -1 and 1.

In [ ]:
def my_process_output(clas_pred, bbox_pred, sizes, detect_thresh = 0.25):
    anchors = create_anchors(sizes, ratios, scales).to(clas_pred.device)
    bbox_pred = activ_to_bbox(bbox_pred, anchors)
    clas_pred = torch.sigmoid(clas_pred)
    detect_mask = clas_pred.max(1)[0] > detect_thresh
    bbox_pred, clas_pred = bbox_pred[detect_mask], clas_pred[detect_mask]
    bbox_pred = tlbr2cthw(torch.clamp(cthw2tlbr(bbox_pred), min=-1, max=1))    
    if clas_pred.numel() == 0: return [],[],[] 
    scores, preds = clas_pred.max(1)
    return bbox_pred, scores, preds

Check single result

In [44]:
out = raw_preds[0][1:]
processed_out = my_process_output(*out, detect_thresh=0.7)
processed_out

(tensor([[-0.3480, -0.4802,  0.4733,  0.7328],
         [-0.3330, -0.4907,  0.6733,  0.5134],
         [-0.3532, -0.2897,  0.4678,  0.7231],
         [-0.3332, -0.3026,  0.6732,  0.5014],
         [-0.3532, -0.1071,  0.4678,  0.7056],
         [-0.3225, -0.1073,  0.6848,  0.4967],
         [-0.3558,  0.1009,  0.4650,  0.7136],
         [-0.3328,  0.0890,  0.6736,  0.4931],
         [-0.3583,  0.3103,  0.4625,  0.7231],
         [-0.3632,  0.4589,  0.4574,  0.6728],
         [-0.1305, -0.4929,  0.4400,  0.7199],
         [-0.1323, -0.4966,  0.6227,  0.5074],
         [-0.1324, -0.3179,  0.5538,  0.8803],
         [-0.1338, -0.3129,  0.4367,  0.7000],
         [-0.1296, -0.3078,  0.6253,  0.4962],
         [-0.1298, -0.1235,  0.4407,  0.6896],
         [-0.1147, -0.1152,  0.6404,  0.4890],
         [-0.1325,  0.0999,  0.4380,  0.7126],
         [-0.1164,  0.0854,  0.6386,  0.4895],
         [-0.1323,  0.2990,  0.4382,  0.7117],
         [-0.1200,  0.2960,  0.6350,  0.5000],
         [-0.

In [45]:
def my_get_predictions(clas_pred, bbox_pred, sizes, detect_thresh=0.05):
    bbox_pred, scores, preds = my_process_output(clas_pred, bbox_pred, sizes, detect_thresh)
    if len(scores) == 0: return [],[],[]
    to_keep = nms(bbox_pred, scores)
    return bbox_pred[to_keep], preds[to_keep], scores[to_keep]

In [46]:
prediction_out = my_get_predictions(*out, detect_thresh=0.7)
prediction_out

(tensor([[-0.1259, -0.3013,  0.4446,  0.7114],
         [ 0.1244, -0.3040,  0.4449,  0.7087],
         [-0.1258,  0.1000,  0.4447,  0.7127],
         [-0.3728, -0.2990,  0.4476,  0.7137],
         [-0.1257,  0.4963,  0.4448,  0.7091],
         [ 0.1237,  0.4962,  0.4441,  0.7090],
         [ 0.1241,  0.0993,  0.4445,  0.7120],
         [-0.3731,  0.1001,  0.4474,  0.7128],
         [ 0.3702, -0.3043,  0.4407,  0.7085],
         [ 0.3705,  0.2993,  0.4409,  0.7120],
         [-0.3738,  0.4959,  0.4466,  0.7086]], device='cuda:0'),
 tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], device='cuda:0'),
 tensor([0.9159, 0.9035, 0.8673, 0.8533, 0.8520, 0.8422, 0.8273, 0.8104, 0.7901,
         0.7408, 0.7359], device='cuda:0'))

## Scale boxes

Get image sizes from dataframe

In [47]:
def get_img_size(fn):
    return tuple(df.loc[df.id == fn.stem, ['training_width', 'training_height']].values[0])

Account for the scaling / translation of the (target) bounding boxes

In [49]:
def correct_bbox(bbox, fn):
    sz = get_img_size(fn)
    tlbr_bbox = cthw2tlbr(bbox).cpu()
    scalar = df.loc[df['id'] == fn.stem, 'scalar'].item()
    scaled_bbox = (tlbr_bbox.view(-1,2) + 1) * tensor(sz) / 2 / scalar
    
    return scaled_bbox.view(-1,4)

Check single bounding box

In [48]:
pred_bbox = prediction_out[0]
pred_bbox

tensor([[-0.1259, -0.3013,  0.4446,  0.7114],
        [ 0.1244, -0.3040,  0.4449,  0.7087],
        [-0.1258,  0.1000,  0.4447,  0.7127],
        [-0.3728, -0.2990,  0.4476,  0.7137],
        [-0.1257,  0.4963,  0.4448,  0.7091],
        [ 0.1237,  0.4962,  0.4441,  0.7090],
        [ 0.1241,  0.0993,  0.4445,  0.7120],
        [-0.3731,  0.1001,  0.4474,  0.7128],
        [ 0.3702, -0.3043,  0.4407,  0.7085],
        [ 0.3705,  0.2993,  0.4409,  0.7120],
        [-0.3738,  0.4959,  0.4466,  0.7086]], device='cuda:0')

In [50]:
fn = raw_preds[0][0]
correct_bbox(pred_bbox, fn)

tensor([[1381.8876,  596.4929, 2324.3721, 1833.7212],
        [1912.1945,  594.1376, 2855.2905, 1826.6897],
        [1382.0240, 1293.2283, 2324.7805, 2532.6733],
        [ 855.1301,  598.5040, 1804.1184, 1839.7529],
        [1382.1067, 1985.6680, 2325.0276, 3218.7754],
        [1911.4065, 1985.5752, 2852.9336, 3218.4980],
        [1911.8453, 1292.6516, 2854.2454, 2530.9441],
        [ 854.8492, 1293.3174, 1803.2708, 2532.9404],
        [2437.6924,  593.9040, 3371.8918, 1825.9941],
        [2438.0012, 1640.4604, 3372.8064, 2878.7454],
        [ 854.0759, 1985.2764, 1800.9396, 3217.6086]])

In [51]:
result = correct_bbox(pred_bbox, fn).tolist(), prediction_out[1].tolist(), prediction_out[2].tolist()
result

([[1381.8875732421875, 596.492919921875, 2324.3720703125, 1833.72119140625],
  [1912.1944580078125, 594.1376342773438, 2855.29052734375, 1826.689697265625],
  [1382.0240478515625, 1293.228271484375, 2324.780517578125, 2532.67333984375],
  [855.130126953125, 598.5040283203125, 1804.118408203125, 1839.7529296875],
  [1382.106689453125, 1985.66796875, 2325.027587890625, 3218.775390625],
  [1911.406494140625, 1985.5751953125, 2852.93359375, 3218.498046875],
  [1911.8453369140625,
   1292.651611328125,
   2854.245361328125,
   2530.944091796875],
  [854.8492431640625, 1293.3173828125, 1803.270751953125, 2532.9404296875],
  [2437.6923828125, 593.9039916992188, 3371.891845703125, 1825.994140625],
  [2438.001220703125, 1640.46044921875, 3372.806396484375, 2878.745361328125],
  [854.075927734375, 1985.2763671875, 1800.9395751953125, 3217.608642578125]],
 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 [0.9158955812454224,
  0.9035295844078064,
  0.8673273324966431,
  0.8532646894454956,
  0.852022945880889

## Get the result in the expected string format

In [52]:
def stringify(x):
    new_list = []
    for item in x:
        if (type(item) != int) and (type(item) != float):
            new_list.append(' '.join(stringify(item)))
        else:            
            new_item = round(item, 5)
            new_list.append(str(new_item))
        
    return new_list

In [54]:
def stringify_all(conf, bbox):
    out_str = ''
    for i in range(len(bbox)):
        if out_str != '':
            out_str += ' '
        out_str += ' '.join(['opacity', conf[i], bbox[i]])
    return out_str

In [53]:
stringify(result[0]), stringify(result[2])

(['1381.88757 596.49292 2324.37207 1833.72119',
  '1912.19446 594.13763 2855.29053 1826.6897',
  '1382.02405 1293.22827 2324.78052 2532.67334',
  '855.13013 598.50403 1804.11841 1839.75293',
  '1382.10669 1985.66797 2325.02759 3218.77539',
  '1911.40649 1985.5752 2852.93359 3218.49805',
  '1911.84534 1292.65161 2854.24536 2530.94409',
  '854.84924 1293.31738 1803.27075 2532.94043',
  '2437.69238 593.90399 3371.89185 1825.99414',
  '2438.00122 1640.46045 3372.8064 2878.74536',
  '854.07593 1985.27637 1800.93958 3217.60864'],
 ['0.9159',
  '0.90353',
  '0.86733',
  '0.85326',
  '0.85202',
  '0.8422',
  '0.82731',
  '0.81044',
  '0.79015',
  '0.74079',
  '0.73588'])

In [55]:
stringify_all(stringify(result[2]), stringify(result[0]))

'opacity 0.9159 1381.88757 596.49292 2324.37207 1833.72119 opacity 0.90353 1912.19446 594.13763 2855.29053 1826.6897 opacity 0.86733 1382.02405 1293.22827 2324.78052 2532.67334 opacity 0.85326 855.13013 598.50403 1804.11841 1839.75293 opacity 0.85202 1382.10669 1985.66797 2325.02759 3218.77539 opacity 0.8422 1911.40649 1985.5752 2852.93359 3218.49805 opacity 0.82731 1911.84534 1292.65161 2854.24536 2530.94409 opacity 0.81044 854.84924 1293.31738 1803.27075 2532.94043 opacity 0.79015 2437.69238 593.90399 3371.89185 1825.99414 opacity 0.74079 2438.00122 1640.46045 3372.8064 2878.74536 opacity 0.73588 854.07593 1985.27637 1800.93958 3217.60864'

## Stringify all predictions and add to submission dataframe

In [56]:
for inst in raw_preds:
    fn = inst[0]
    bbox_pred, class_pred, confidence = my_get_predictions(*inst[1:], detect_thresh=0.37)
    if bbox_pred == []:
        label = 'none 1 0 0 1 1'
    else:
        bbox_pred = correct_bbox(bbox_pred, fn)
        label = stringify_all(stringify(confidence.tolist()), stringify(bbox_pred.tolist()))
    sub_df.loc[sub_df.id == fn.stem + '_image', 'PredictionString'] = label

In [57]:
sub_df.tail()

,id,PredictionString
2472,46719b856de1_image,opacity 0.8774 1297.4364 222.17497 1944.14087 1234.1748 opacity 0.85458 1617.04724 438.85425 2192.74561 1351.2229 opacity 0.83221 931.4082 470.35956 1562.78418 1445.8335 opacity 0.71405 1150.45398 846.02155 2032.57642 1490.96436 opacity 0.69856 1614.67175 961.55011 2186.49463 1861.94812 opacity 0.67312 2028.27991 454.76828 2705.29565 1397.95715 opacity 0.67053 589.90082 714.86987 1256.09387 1649.177 opacity 0.65762 1273.77625 1245.8418 1875.29187 2183.7146 opacity 0.55154 926.67383 1249.36707 1548.92322 2194.26831 opacity 0.54013 1612.73975 1478.95276 2181.44531 2358.33374 opacity 0.53765 ...
2473,31c07523a69a_image,opacity 0.91916 1381.58301 1961.95129 2316.75439 3143.62573 opacity 0.91642 1915.63318 923.98267 2854.81323 2120.62891 opacity 0.91549 1907.21143 1971.75159 2830.1377 3171.45264 opacity 0.90716 1384.5907 953.19562 2325.6543 2206.61304 opacity 0.83291 2400.31445 590.24719 3254.66748 1815.34424 opacity 0.82991 874.11462 1655.15503 1859.62939 2917.88184 opacity 0.81719 2399.42651 1269.83264 3252.3252 2461.19067 opacity 0.77894 881.73834 639.39874 1884.05542 1968.26599 opacity 0.72519 1721.2207 436.30209 3056.79248 1322.81372 opacity 0.64678 3013.98389 926.4093 4025.92212 2127.57837 opacity 0....
2474,f77d7d1aebab_image,opacity 0.75153 977.83948 1116.2207 1639.82959 2168.49487 opacity 0.70662 615.20953 814.73853 1305.36755 1873.69885 opacity 0.69357 977.9021 1699.32971 1640.01453 2703.27954 opacity 0.67781 1351.60193 782.43219 2008.30176 1783.25745 opacity 0.66581 616.78229 1437.70654 1310.28894 2521.78027 opacity 0.6427 985.5542 498.44318 1662.91882 1536.40503 opacity 0.6176 1351.91138 1697.06653 2009.2052 2697.14648 opacity 0.54528 1707.46387 207.4668 2326.76147 1271.72534 opacity 0.51757 199.93159 1125.1842 816.61218 2194.5415 opacity 0.51365 1695.5946 1100.75793 2295.13965 2124.76953 opacity 0.46153 2...
2475,ccc5b63ca96d_image,opacity 0.7855 1362.75305 692.44763 2028.01221 1605.69824 opacity 0.78096 1718.42419 459.86502 2341.18799 1415.15833 opacity 0.73182 1371.20044 1201.11072 2053.271 2124.21875 opacity 0.67684 2136.76343 464.37109 2834.63086 1430.02563 opacity 0.6423 995.45709 1206.5531 1684.23938 2141.32422 opacity 0.64025 991.25928 694.03052 1671.41504 1610.56873 opacity 0.63387 1720.9043 1426.76416 2347.93921 2297.38379 opacity 0.61538 1552.76514 1057.09216 2410.07227 1670.19019 opacity 0.58695 1221.04041 321.38852 2162.36011 974.68579 opacity 0.53014 2140.34448 1182.78076 2845.95728 2068.69849 opacity 0....
2476,5e8ac1fe2b82_image,opacity 0.91229 1364.66577 457.0701 2033.65881 1406.0553 opacity 0.89962 1713.48425 673.6225 2327.89917 1549.51489 opacity 0.84141 1353.8291 1194.21912 2002.19397 2102.98071 opacity 0.83789 985.97754 686.9989 1655.58325 1589.11328 opacity 0.8098 2150.25488 682.17987 2878.28149 1574.67004 opacity 0.80219 979.01233 1200.39783 1635.19141 2122.00073 opacity 0.73078 1569.2113 332.16068 2454.09253 1009.14911 opacity 0.71764 1714.28931 1409.94238 2330.05078 2249.9729 opacity 0.61182 631.07056 951.61743 1351.73083 1880.00769 opacity 0.57573 843.64539 319.12845 1788.09424 967.67218 opacity 0.55958 ...


## Save dataframe to .csv

In [ ]:
sub_df.to_csv('submission.csv', index = False)